In [1]:
def pnn(imname):
    import matplotlib.pyplot as plt
    import skimage.io
    from skimage.filters import threshold_mean
    from skimage.morphology import remove_small_objects

    img = skimage.io.imread(imname) #the image needs to be in the same folder as the code

    pnn = img[:, :, 1]

    thresh = threshold_mean(pnn)
    pnnbinary = pnn > thresh
    
    from skimage.filters.rank import median
    from skimage.morphology import disk
    from scipy import ndimage as ndi

    fillpnn = median(ndi.binary_fill_holes(pnnbinary), disk(2))
    
    threshfill = threshold_mean(fillpnn)
    fillpnnbinary = fillpnn > threshfill

    pnnclim = remove_small_objects(fillpnnbinary, min_size=3000)
    
    return [pnnclim]

In [2]:
pnnclim = pnn('HBBS_vid_1_overlay_frame_1.tif')

In [3]:
def listparticles(imname):
    import matplotlib.pyplot as plt
    import skimage.io
    from skimage.filters import threshold_yen
    import numpy as np
    import math as m

    img = skimage.io.imread(imname) #the image needs to be in the same folder as the code

    #channel 0 = particles
    partim = img[:, :, 0]

    #binary image
    thresh = threshold_yen(partim)
    partbinary = partim > thresh

    #label each object in the image
    partlbim = skimage.measure.label(partbinary)

    #set properties
    props = skimage.measure.regionprops(partlbim)

    numobj = partlbim.max()

    particles = np.zeros((numobj, 2)) #rows, columns

    i = 0 
    while i < numobj:
        particles[i,0] = props[i].centroid[0] #x-coordinate
        particles[i,1] = props[i].centroid[1] #y-coordinate
        i = i + 1
    
    return particles

In [4]:
xyparticles = listparticles('HBBS_vid_1_overlay_frame_1.tif')

In [5]:
xyparticles

array([[ 50.63218391,  86.73563218],
       [ 86.99438202, 332.84269663],
       [ 96.66666667, 326.33333333],
       [106.48148148, 335.18518519],
       [157.54545455,  80.36363636],
       [207.875     , 368.875     ],
       [234.08333333, 179.33333333],
       [247.66666667, 281.66666667],
       [296.53191489, 244.10638298],
       [329.        , 387.54545455]])

In [6]:
def partbin(particles, pnnclim):
    import numpy as np
    
    partdistance = np.zeros((len(particles), 4)) #array with x, and y coordinates of particles and the distance to the cells
    
    from scipy import ndimage as ndi
    distance = ndi.distance_transform_edt(np.invert(pnnclim))[0,:,:]
    
    for i in range(0,len(particles)): #need parenthese not brackets
        partdistance[i,0] = int(particles[i,0])
        partdistance[i,1] = int(particles[i,1])
        partdistance[i,2] = distance[int(particles[i,0]),int(particles[i,1])] #force int
        if partdistance[i,2] == 0:
            partdistance[i,3] = 1
        elif partdistance[i,2] > 0 and partdistance[i,2] <= 20:
            partdistance[i,3] = 2
        elif partdistance[i,2] > 20 and partdistance[i,2] <= 80: 
            partdistance[i,3] = 3
        else:
            partdistance[i,3] = 4

    return [partdistance]

In [7]:
partbin(xyparticles, pnnclim)

[array([[ 50.        ,  86.        , 105.36128321,   4.        ],
        [ 86.        , 332.        ,  42.48529157,   3.        ],
        [ 96.        , 326.        ,  30.87069808,   3.        ],
        [106.        , 335.        ,  28.31960452,   3.        ],
        [157.        ,  80.        ,  85.0940656 ,   4.        ],
        [207.        , 368.        ,  54.4242593 ,   3.        ],
        [234.        , 179.        ,  13.60147051,   2.        ],
        [247.        , 281.        ,   1.        ,   2.        ],
        [296.        , 244.        ,   2.23606798,   2.        ],
        [329.        , 387.        ,  87.32124598,   4.        ]])]